In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import random as random
import math as math
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import os
import pickle

In [14]:
train = pd.read_csv("./data/authors_with_clusters_final.csv")
train = train.drop(columns=['Unnamed: 0', 'num_citations', 'keywords', 'affiliation', 'citations'])
train = train.dropna(axis = 0, subset=['co_authors'])
train = train[train.co_authors != '[]']
train['author_id'] = pd.to_numeric(train['author_id'])
train.head()

,Unnamed: 0.1,author_id,co_authors,h_index,paper_count,cluster_id
0,Jiawei_Han,40255,"[42167, 11017, 44161, 16038, 64503, 64504, 645...",144,737,-2
1,Anil_Jain,45287,"[45286, 66718, 66719, 67047, 67048, 67049, 787...",144,549,-2
2,Philip_Yu,738,"[2492, 9499, 9500, 13689, 13690, 24400, 16425,...",132,976,-1
3,Andrew_Zisserman,7015,"[5776, 46602, 96985, 31335, 81469, 100220, 100...",123,360,-3
4,Thomas_Huang,15850,"[15849, 27280, 27281, 45764, 82995, 88292, 640...",122,827,-3


In [15]:
def change_type(authors_list):
    return list(map(int, authors_list[1:-1].split(',')))
    
train['co_authors'] = train['co_authors'].apply(change_type)

KeyboardInterrupt: 

In [ ]:
train = train[train.cluster_id != 9999999]

In [ ]:
train.rename( columns={'Unnamed: 0.1':'author_name'}, inplace=True )

In [13]:
train.head()

,author_name,author_id,co_authors,cluster_id
0,Jiawei_Han,40255,"[42167, 11017, 44161, 16038, 64503, 64504, 645...",-2
1,Anil_Jain,45287,"[45286, 66718, 66719, 67047, 67048, 67049, 787...",-2
2,Philip_Yu,738,"[2492, 9499, 9500, 13689, 13690, 24400, 16425,...",-1
3,Andrew_Zisserman,7015,"[5776, 46602, 96985, 31335, 81469, 100220, 100...",-3
4,Thomas_Huang,15850,"[15849, 27280, 27281, 45764, 82995, 88292, 640...",-3


In [7]:
cluster_grp_data = train.groupby(train['cluster_id'])

In [8]:
from matplotlib.pyplot import figure
def show_graph(G):
    figure(num=None, figsize=(32, 24), dpi=80, facecolor='w', edgecolor='k')
    pos = nx.random_layout(G)
    nx.draw_networkx_nodes(G,pos,alpha=0.4)
    nx.draw_networkx_edges(G,pos,width=1,alpha=0.4)
    plt.show()

Graph_list = []
pagerank_dict = {}
for cluster, df in cluster_grp_data:
    G = nx.Graph()
    for i in range(len(df)):
        auth = df.iloc[i]['author_id']
        for neighbor in df.iloc[i]['co_authors']:
            if G.has_edge(auth, neighbor):
                G.add_edge(auth, neighbor, weight = G[auth][neighbor]['weight']+1)
            else:
                G.add_edge(auth, neighbor, weight = 1)
    score = nx.pagerank(G, alpha=0.55, tol=1.0e-6, nstart=None, weight='weight', dangling=None)
    pagerank_dict[cluster] = score
    

{-14: {45005: 0.000489471372900731,
  45004: 1.147771453698072e-05,
  59123: 7.876897404082774e-06,
  59124: 7.876897404082774e-06,
  265888: 7.876897404082774e-06,
  262539: 1.1816057888288382e-05,
  265889: 7.876897404082774e-06,
  320613: 7.876897404082774e-06,
  317509: 1.7554391208760036e-05,
  325780: 7.876897404082774e-06,
  329382: 7.876897404082774e-06,
  339676: 7.876897404082774e-06,
  341952: 7.876897404082774e-06,
  101802: 2.036453463299876e-05,
  224556: 7.876897404082774e-06,
  360319: 9.568605095900954e-06,
  238358: 7.876897404082774e-06,
  360318: 1.4482396408757166e-05,
  433648: 7.876897404082774e-06,
  456751: 7.876897404082774e-06,
  475448: 7.876897404082774e-06,
  507091: 7.876897404082774e-06,
  40365: 1.9497983093099996e-05,
  122846: 1.1407325410856841e-05,
  268663: 1.4017109268749668e-05,
  407388: 1.6581325887618295e-05,
  201535: 1.756546651932534e-05,
  44766: 9.548853811994529e-06,
  104373: 4.2137839820841016e-05,
  280662: 0.00033005495571428607,
  1

In [11]:
with open('./data/author_page_rank.pickle', 'wb') as handle:
    pickle.dump(pagerank_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
from matplotlib.pyplot import figure

def show_graph(G):
    figure(num=None, figsize=(32, 24), dpi=80, facecolor='w', edgecolor='k')
    pos = nx.random_layout(G)
    nx.draw_networkx_nodes(G,pos,alpha=0.4)
    nx.draw_networkx_edges(G,pos,width=1,alpha=0.4)
    plt.show()
    
G1 = nx.Graph()
for i in range(1000000):
    for neighbor in train.iloc[i]['references']:
            G1.add_edge(train.iloc[i]['id'], neighbor)
            
# show_graph(G1)

In [ ]:
for name,group in cluster_grp_data:
    print(name)
    print(type(group))

In [ ]:
len(train)

In [ ]:
train.cluster_id

In [ ]:
t